In [3]:
import sys
import json

general = {'duration': 24 * 60}
params = {'general': general}

default = {'training_folder': '../../', 'scenario': 'montreal_periods', 'period': '', 'params': params}
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../', 'scenario': 'montreal_periods', 'period': '', 'params': {'general': {'duration': 1440}}}


In [4]:
import os
import geopandas as gpd
import pandas as pd
from copy import deepcopy

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('on_lambda', on_lambda)


on_lambda False


# Folders stucture and params

In [ ]:
scenario = argv['scenario']
training_folder = argv['training_folder']

# if local. add the path to the scenario scenarios/<scenario>/
local_scen_path = '' if on_lambda else os.path.join('scenarios/', scenario)

input_folder = os.path.join(training_folder, 'inputs/')
scenario_folder = os.path.join(training_folder, local_scen_path, 'inputs/')
model_folder = os.path.join(training_folder, local_scen_path, 'model/')
output_folder = os.path.join(training_folder, local_scen_path, 'outputs/')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print('input folder: ', scenario_folder)
print('output folder: ', output_folder)

input folder:  ../../scenarios/montreal_periods/inputs/
output folder:  ../../scenarios/montreal_periods/outputs/


In [6]:
periods = [d for d in os.listdir(output_folder) if os.path.isdir(os.path.join(output_folder, d))]
print(periods)

['pm', 'am']


In [7]:
# os.listdir(os.path.join(output_folder, 'am'))

In [ ]:
def merge_periods_and_save(periods, filename='nodes.geojson', cols=['catchment', 'catchment_radius']):
    merged_df = pd.DataFrame()
    for period in periods:
        filepath = os.path.join(output_folder, period, filename)
        df = gpd.read_file(filepath)
        df = df.set_index('index')

        var_cols = [f'{col}#{period}' for col in cols]
        names = {col: var_col for col, var_col in zip(cols, var_cols)}
        df = df.rename(columns=names)
        if len(merged_df) == 0:
            merged_df = df
        else:
            merged_df = merged_df.merge(df[var_cols], left_index=True, right_index=True)

    if len(periods) > 0:
        merged_df.to_file(os.path.join(output_folder, filename), driver='GeoJSON')

In [9]:
merge_periods_and_save(periods, 'nodes.geojson', ['catchment', 'catchment_radius'])

In [10]:
merge_periods_and_save(periods, 'population_mesh.geojson', ['population'])

# Styles

In [ ]:
with open(os.path.join(input_folder, 'styles.json')) as f:
    templates = json.load(f)

In [ ]:
if len(periods) == 0:
    with open(os.path.join(output_folder, 'styles.json'), 'w') as f:
        json.dump(templates, f)
    end_of_notebook

In [13]:
styles = []

In [ ]:
template = [s for s in templates if s['name'] == 'population mesh'][0]
for period in periods:
    style = deepcopy(template)
    style['name'] = f'{style["name"]} {period}'
    feature = style['displaySettings']['selectedFeature']
    style['displaySettings']['selectedFeature'] = f'{feature}#{period}'
    styles.append(style)


In [ ]:
template = [s for s in templates if s['name'] == 'stop catchment'][0]
for period in periods:
    style = deepcopy(template)
    style['name'] = f'{style["name"]} {period}'
    feature = style['displaySettings']['selectedFeature']
    style['displaySettings']['selectedFeature'] = f'{feature}#{period}'
    styles.append(style)

In [ ]:
template = [s for s in templates if s['name'] == 'OD routes'][0]
for period in periods:
    style = deepcopy(template)
    filename = os.path.join(output_folder, period, 'od_route.geojson')
    if os.path.isfile(filename):
        style['name'] = f'{style["name"]} {period}'
        style['layer'] = f'microservices/transit/{period}/od_route'
        styles.append(style)

In [ ]:
template = [s for s in templates if s['name'] == 'isochrone'][0]
for period in periods:
    style = deepcopy(template)
    filename = os.path.join(output_folder, period, 'isochrone.geojson')
    if os.path.isfile(filename):
        style['name'] = f'{style["name"]} {period}'
        style['layer'] = f'microservices/transit/{period}/isochrone'
        styles.append(style)

In [ ]:
with open(os.path.join(output_folder, 'styles.json'), 'w') as f:
    json.dump(styles, f)